In [141]:
"""
Script per caricare il dataset di MangaDex in JupyterLab utilizzando Pandas.
Lo script legge il file CSV, visualizza le prime righe del dataframe
e fornisce una panoramica delle colonne presenti per facilitare l'analisi successiva.
"""
import pandas as pd
mangadex_df = pd.read_csv("mangadex_normalized.csv")

mangadex_df.head()


manga_id  \
0  42c57bae-fc53-4aa1-b2e3-0c51e30b0521   
1  39ec041f-5bc7-46f5-a2e7-4af04fc35869   
2  3a7bd813-c6ea-4052-a6a9-6a5e617e27a6   
3  a5f502a7-9ab0-48de-9dec-c7b7bf54b905   
4  51ebd014-189e-4ff4-bac8-968ff0c2b094   

                                          title_api  \
0                            #TodaysUnderwearReport   
1  'Kore Ageru kara sa; Onee-san to Asonde kureru?'   
2                       +99 Reinforced Wooden Stick   
3                                            -Rain-   
4                                            .traeH   

                                          alt_titles          author  \
0  何故か毎日下着の色を報告してくるガールズ | Nazeka Mainichi Shitagi...         Dojirou   
1  「これあげるからさ、お姉さんと遊んでくれる？」 | Te daré esto ¿Jugará...   Nishizawa 5mm   
2  99강화나무몽둥이 | 99ganghwanamumongdungi | 99 Reinfo...   Hong-Sil (홍실)   
3           Rain (HAYASE Hashiba) | -RAIN- (-ДОЖДЬ-)  Hayase Hashiba   
4                             .قلب | traeH | .ецдреС    Hanatsu Yaya   

           artist demographic     status    year  \
0         Dojirou     shounen    ongoing  2020.0   
1   Nishizawa 5mm         NaN  completed  2020.0   
2    Jiperi (지페리)         NaN    ongoing  2022.0   
3  Hayase Hashiba         NaN  completed     NaN   
4    Hanatsu Yaya         NaN  completed     NaN   

                                         description content_rating  \
0  Girls who really want to show off their underw...     suggestive   
1  It's literally an advertisement for a figure w...     suggestive   
2  Welcome to “Chrono Life;” a virtual reality ga...           safe   
3  On a rainy day; when Mai got stood up by the g...        erotica   
4  After being abandoned by first her father then...           safe   

       last_updated  follows  rating  \
0  05/03/2024 09:10    21427  7.6169   
1  13/04/2023 22:07     1483  6.6437   
2  29/09/2023 02:30     3358  8.2788   
3  10/12/2024 21:39      381  6.2049   
4  05/03/2023 16:03      312  7.4545   

                                 title_normalized  
0                           todaysunderwearreport  
1  'kore ageru kara sa onee-san to asonde kureru'  
2                      99 reinforced wooden stick  
3                                          -rain-  
4                                           traeh

In [142]:
mangadex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   manga_id          2998 non-null   object 
 1   title_api         2998 non-null   object 
 2   alt_titles        2884 non-null   object 
 3   author            2998 non-null   object 
 4   artist            2987 non-null   object 
 5   demographic       1743 non-null   object 
 6   status            2998 non-null   object 
 7   year              2274 non-null   float64
 8   description       2791 non-null   object 
 9   content_rating    2998 non-null   object 
 10  last_updated      2998 non-null   object 
 11  follows           2998 non-null   int64  
 12  rating            2998 non-null   float64
 13  title_normalized  2998 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 328.0+ KB


In [143]:
"""
Normalizzazione preliminare della colonna 'alt_titles' in MangaDex.
Rimuove caratteri speciali, converte in minuscolo e pulisce spazi multipli,
ma mantiene le lingue non latine per il matching successivo.
"""

import re

# Rimozione degli spazi multipli e caratteri speciali, mantenendo i caratteri non latini
mangadex_df["alt_titles"] = mangadex_df["alt_titles"].str.lower()
mangadex_df["alt_titles"] = mangadex_df["alt_titles"].str.replace(r"[^\w\s\|’'”“‘]", "", regex=True)
mangadex_df["alt_titles"] = mangadex_df["alt_titles"].str.replace(r"\s+", " ", regex=True).str.strip()

mangadex_df["alt_titles"].head(10)


0    何故か毎日下着の色を報告してくるガールズ | nazeka mainichi shitagi...
1    これあげるからさお姉さんと遊んでくれる | te daré esto jugarás con...
2    99강화나무몽둥이 | 99ganghwanamumongdungi | 99 reinfo...
3                     rain hayase hashiba | rain дождь
4                                 قلب | traeh | ецдрес
5    0c magic | 0の魔法 | ma magie par 0c | zero degre...
6    1 cân fang fập fóng fụt 9 | 1らぶ9 | １９ | 19 イチら...
7    1年c組吉田係 | first year class c yoshida represent...
8    12シガレット | half cigarette | nửa điếu thuốc lá |...
9    12 love flower of love | 12 love flower of lov...
Name: alt_titles, dtype: object

In [144]:
"""
Script per identificare le lingue dei titoli alternativi nella colonna 'alt_titles' di MangaDex.
Separa i titoli usando la barra verticale e organizza i titoli per lingua in nuove colonne.
"""

from lingua import Language, LanguageDetectorBuilder


detector = LanguageDetectorBuilder.from_all_languages().build()

def split_and_detect_languages(alt_titles):
    if pd.isna(alt_titles):
        return {}
    
    titles = [title.strip() for title in alt_titles.split('|') if title.strip()]
    lang_dict = {}
    
    for title in titles:
        try:
            language = detector.detect_language_of(title)
            lang_name = language.name.lower()
            if lang_name not in lang_dict:
                lang_dict[lang_name] = []
            lang_dict[lang_name].append(title)
        except:
            continue
            
    return lang_dict

mangadex_df["alt_titles_by_language"] = mangadex_df["alt_titles"].apply(split_and_detect_languages)
mangadex_df[["alt_titles", "alt_titles_by_language"]].head(10)



alt_titles  \
0  何故か毎日下着の色を報告してくるガールズ | nazeka mainichi shitagi...   
1  これあげるからさお姉さんと遊んでくれる | te daré esto jugarás con...   
2  99강화나무몽둥이 | 99ganghwanamumongdungi | 99 reinfo...   
3                   rain hayase hashiba | rain дождь   
4                               قلب | traeh | ецдрес   
5  0c magic | 0の魔法 | ma magie par 0c | zero degre...   
6  1 cân fang fập fóng fụt 9 | 1らぶ9 | １９ | 19 イチら...   
7  1年c組吉田係 | first year class c yoshida represent...   
8  12シガレット | half cigarette | nửa điếu thuốc lá |...   
9  12 love flower of love | 12 love flower of lov...   

                              alt_titles_by_language  
0  {'japanese': ['何故か毎日下着の色を報告してくるガールズ', '本日の下着報告...  
1  {'japanese': ['これあげるからさお姉さんと遊んでくれる'], 'malay':...  
2  {'korean': ['99강화나무몽둥이'], 'latin': ['99ganghwa...  
3  {'shona': ['rain hayase hashiba'], 'russian': ...  
4  {'arabic': ['قلب'], 'maori': ['traeh'], 'mongo...  
5  {'latin': ['0c magic'], 'japanese': ['0の魔法'], ...  
6  {'shona': ['1 cân fang fập fóng fụt 9', 'ichi ...  
7  {'chinese': ['1年c組吉田係'], 'english': ['first ye...  
8  {'japanese': ['12シガレット'], 'english': ['half ci...  
9  {'english': ['12 love flower of love', '12 lov...

In [ ]:
"""
Separazione degli alt_titles in base alla lingua nel dataset di MangaDex.
Separa i titoli usando la barra verticale e crea colonne separate per ogni lingua.
"""

def split_and_detect_languages(alt_titles):
    if pd.isna(alt_titles):
        return {}
    
    titles = [title.strip() for title in alt_titles.split('|') if title.strip()]
    lang_dict = {}
    
    for title in titles:
        try:
            language = detector.detect_language_of(title)
            lang_name = language.name.lower()
            if lang_name not in lang_dict:
                lang_dict[lang_name] = []
            lang_dict[lang_name].append(title)
        except Exception as e:
            continue
            
    return lang_dict

mangadex_df["alt_titles_by_language"] = mangadex_df["alt_titles"].apply(split_and_detect_languages)

alt_titles_by_language_df = pd.json_normalize(mangadex_df["alt_titles_by_language"])

mangadex_df = pd.concat([mangadex_df, alt_titles_by_language_df], axis=1)

mangadex_df.head()



manga_id  \
0  42c57bae-fc53-4aa1-b2e3-0c51e30b0521   
1  39ec041f-5bc7-46f5-a2e7-4af04fc35869   
2  3a7bd813-c6ea-4052-a6a9-6a5e617e27a6   
3  a5f502a7-9ab0-48de-9dec-c7b7bf54b905   
4  51ebd014-189e-4ff4-bac8-968ff0c2b094   

                                          title_api  \
0                            #TodaysUnderwearReport   
1  'Kore Ageru kara sa; Onee-san to Asonde kureru?'   
2                       +99 Reinforced Wooden Stick   
3                                            -Rain-   
4                                            .traeH   

                                          alt_titles          author  \
0  何故か毎日下着の色を報告してくるガールズ | nazeka mainichi shitagi...         Dojirou   
1  これあげるからさお姉さんと遊んでくれる | te daré esto jugarás con...   Nishizawa 5mm   
2  99강화나무몽둥이 | 99ganghwanamumongdungi | 99 reinfo...   Hong-Sil (홍실)   
3                   rain hayase hashiba | rain дождь  Hayase Hashiba   
4                               قلب | traeh | ецдрес    Hanatsu Yaya   

           artist demographic     status    year  \
0         Dojirou     shounen    ongoing  2020.0   
1   Nishizawa 5mm         NaN  completed  2020.0   
2    Jiperi (지페리)         NaN    ongoing  2022.0   
3  Hayase Hashiba         NaN  completed     NaN   
4    Hanatsu Yaya         NaN  completed     NaN   

                                         description content_rating  ...  \
0  Girls who really want to show off their underw...     suggestive  ...   
1  It's literally an advertisement for a figure w...     suggestive  ...   
2  Welcome to “Chrono Life;” a virtual reality ga...           safe  ...   
3  On a rainy day; when Mai got stood up by the g...        erotica  ...   
4  After being abandoned by first her father then...           safe  ...   

  croatian  bengali  kazakh estonian icelandic latvian czech tamil urdu telugu  
0      NaN      NaN     NaN      NaN       NaN     NaN   NaN   NaN  NaN    NaN  
1      NaN      NaN     NaN      NaN       NaN     NaN   NaN   NaN  NaN    NaN  
2      NaN      NaN     NaN      NaN       NaN     NaN   NaN   NaN  NaN    NaN  
3      NaN      NaN     NaN      NaN       NaN     NaN   NaN   NaN  NaN    NaN  
4      NaN      NaN     NaN      NaN       NaN     NaN   NaN   NaN  NaN    NaN  

[5 rows x 87 columns]

In [146]:
"""
Normalizzazione delle colonne 'author' e 'artist' nel dataset di MangaDex.
Rimuove caratteri speciali, converte in minuscolo e pulisce spazi multipli,
ma mantiene le lingue non latine per un matching più accurato.
"""

def normalize_names(name):
    if pd.isna(name):
        return ""
    
    name = name.lower()
    name = re.sub(r"[^\w\s\-']", "", name)
    name = re.sub(r"\s+", " ", name).strip()
    
    return name

mangadex_df["author_normalized"] = mangadex_df["author"].apply(normalize_names)
mangadex_df["artist_normalized"] = mangadex_df["artist"].apply(normalize_names)

mangadex_df.head()


manga_id  \
0  42c57bae-fc53-4aa1-b2e3-0c51e30b0521   
1  39ec041f-5bc7-46f5-a2e7-4af04fc35869   
2  3a7bd813-c6ea-4052-a6a9-6a5e617e27a6   
3  a5f502a7-9ab0-48de-9dec-c7b7bf54b905   
4  51ebd014-189e-4ff4-bac8-968ff0c2b094   

                                          title_api  \
0                            #TodaysUnderwearReport   
1  'Kore Ageru kara sa; Onee-san to Asonde kureru?'   
2                       +99 Reinforced Wooden Stick   
3                                            -Rain-   
4                                            .traeH   

                                          alt_titles          author  \
0  何故か毎日下着の色を報告してくるガールズ | nazeka mainichi shitagi...         Dojirou   
1  これあげるからさお姉さんと遊んでくれる | te daré esto jugarás con...   Nishizawa 5mm   
2  99강화나무몽둥이 | 99ganghwanamumongdungi | 99 reinfo...   Hong-Sil (홍실)   
3                   rain hayase hashiba | rain дождь  Hayase Hashiba   
4                               قلب | traeh | ецдрес    Hanatsu Yaya   

           artist demographic     status    year  \
0         Dojirou     shounen    ongoing  2020.0   
1   Nishizawa 5mm         NaN  completed  2020.0   
2    Jiperi (지페리)         NaN    ongoing  2022.0   
3  Hayase Hashiba         NaN  completed     NaN   
4    Hanatsu Yaya         NaN  completed     NaN   

                                         description content_rating  ...  \
0  Girls who really want to show off their underw...     suggestive  ...   
1  It's literally an advertisement for a figure w...     suggestive  ...   
2  Welcome to “Chrono Life;” a virtual reality ga...           safe  ...   
3  On a rainy day; when Mai got stood up by the g...        erotica  ...   
4  After being abandoned by first her father then...           safe  ...   

  kazakh  estonian  icelandic latvian czech tamil urdu telugu  \
0    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
1    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
2    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
3    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
4    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   

  author_normalized artist_normalized  
0           dojirou           dojirou  
1     nishizawa 5mm     nishizawa 5mm  
2       hong-sil 홍실        jiperi 지페리  
3    hayase hashiba    hayase hashiba  
4      hanatsu yaya      hanatsu yaya  

[5 rows x 89 columns]

In [147]:

for col in alt_titles_by_language_df.columns:
    alt_titles_by_language_df[col] = alt_titles_by_language_df[col].apply(lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else x)

mangadex_df.update(alt_titles_by_language_df)

alt_titles_by_language_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 72 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   japanese     2323 non-null   object
 1   shona        332 non-null    object
 2   english      1357 non-null   object
 3   chinese      1202 non-null   object
 4   xhosa        89 non-null     object
 5   tswana       99 non-null     object
 6   malay        117 non-null    object
 7   korean       667 non-null    object
 8   latin        210 non-null    object
 9   slovak       17 non-null     object
 10  thai         270 non-null    object
 11  russian      767 non-null    object
 12  arabic       203 non-null    object
 13  maori        77 non-null     object
 14  mongolian    57 non-null     object
 15  german       160 non-null    object
 16  hindi        36 non-null     object
 17  polish       114 non-null    object
 18  slovene      18 non-null     object
 19  swahili      325 non-null  

In [ ]:
"""
Rimozione dei duplicati interni e dei titoli troppo simili nelle colonne linguistiche
generate dalla separazione con Lingua.
"""

from difflib import SequenceMatcher

SIMILARITY_THRESHOLD = 0.85

def remove_duplicates_and_similar(titles):
    if not isinstance(titles, list):
        return titles
    
    unique_titles = list(dict.fromkeys(titles))
    
    final_titles = []
    for title in unique_titles:
        if not any(SequenceMatcher(None, title, t).ratio() > SIMILARITY_THRESHOLD for t in final_titles):
            final_titles.append(title)
    
    return final_titles

for col in alt_titles_by_language_df.columns:
    alt_titles_by_language_df[col] = alt_titles_by_language_df[col].apply(remove_duplicates_and_similar)

mangadex_df.update(alt_titles_by_language_df)

alt_titles_by_language_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 72 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   japanese     2323 non-null   object
 1   shona        332 non-null    object
 2   english      1357 non-null   object
 3   chinese      1202 non-null   object
 4   xhosa        89 non-null     object
 5   tswana       99 non-null     object
 6   malay        117 non-null    object
 7   korean       667 non-null    object
 8   latin        210 non-null    object
 9   slovak       17 non-null     object
 10  thai         270 non-null    object
 11  russian      767 non-null    object
 12  arabic       203 non-null    object
 13  maori        77 non-null     object
 14  mongolian    57 non-null     object
 15  german       160 non-null    object
 16  hindi        36 non-null     object
 17  polish       114 non-null    object
 18  slovene      18 non-null     object
 19  swahili      325 non-null  

In [149]:
mangadex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 89 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   manga_id                2998 non-null   object 
 1   title_api               2998 non-null   object 
 2   alt_titles              2884 non-null   object 
 3   author                  2998 non-null   object 
 4   artist                  2987 non-null   object 
 5   demographic             1743 non-null   object 
 6   status                  2998 non-null   object 
 7   year                    2274 non-null   float64
 8   description             2791 non-null   object 
 9   content_rating          2998 non-null   object 
 10  last_updated            2998 non-null   object 
 11  follows                 2998 non-null   int64  
 12  rating                  2998 non-null   float64
 13  title_normalized        2998 non-null   object 
 14  alt_titles_by_language  2998 non-null   

In [150]:
mangadex_df.to_csv("mangadex_optimized.csv", index=False)
